### Load dataset

In [28]:
import pandas as pd
RANDOM_STATE = 42

%run "DataHelpers.ipynb"

# Can be replaced with desired variant for different feature sets
GENE_FILE_VARIANT = FeatureVariant.RESEARCH_LOG # For values, see FeatureVariant.print_info()
variant = ModelVariant.RF                      # For values, see ModelVariant.print_info()

FILE_PATH = f"../Data/patient_genes_{GENE_FILE_VARIANT}.csv"

df = pd.read_csv(FILE_PATH)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 977 entries, 0 to 976
Data columns (total 33 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   FYN      977 non-null    float64
 1   BRCA1    977 non-null    float64
 2   FOXC1    977 non-null    float64
 3   TBC1D1   977 non-null    float64
 4   LAG3     977 non-null    float64
 5   CDK6     977 non-null    float64
 6   GATA3    977 non-null    float64
 7   CCND1    977 non-null    float64
 8   PRR4     977 non-null    float64
 9   EPCAM    977 non-null    float64
 10  CD274    977 non-null    float64
 11  PIK3CA   977 non-null    float64
 12  TOP2A    977 non-null    float64
 13  DCLK1    977 non-null    float64
 14  MYC      977 non-null    float64
 15  LRPPRC   977 non-null    float64
 16  BRCA2    977 non-null    float64
 17  TP53     977 non-null    float64
 18  EGFR     977 non-null    float64
 19  MKI67    977 non-null    float64
 20  TTN      977 non-null    float64
 21  CTLA4    977 non

### Import model function

In [29]:
from sklearn.ensemble import RandomForestClassifier

### Dataset split: training and test data

In [30]:
X, y, X_train, X_test, y_train, y_test, test_case_ids = split_data(df, "tnbc", True)
print("\nApplied Smote")
X_smote, y_smote, X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote = split_data_apply_smote(df, "tnbc")

X_train.shape=(781, 31)
X_test.shape=(196, 31)
y_train.shape=(781,)
y_test.shape=(196,)

Applied Smote
X_train.shape=(1379, 31)
X_test.shape=(345, 31)
y_train.shape=(1379,)
y_test.shape=(345,)


### Random Forest

In [31]:
# Create model

model = RandomForestClassifier(random_state=RANDOM_STATE)

def run_model(X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series, test_case_ids, is_smote: bool):
    # Train the model
    model.fit(X_train, y_train)

    # Model predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]  # For ROC curves etc.

    # Save it in a dataframe, to CSV
    predictions = pd.DataFrame({
        "case_id": test_case_ids,
        "y_test": y_test,
        "y_pred": y_pred,
        "y_prob": y_prob
    })
    predictions.to_csv(f"../Data/model_output_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)

    return y_pred, y_prob

In [32]:
y_pred, y_prod = run_model(X_train, X_test, y_train, y_test, test_case_ids, False)

print_evaluated_model_accuracy(y_test, y_pred)

Accuracy: 0.95


## Smote applied

In [33]:
y_pred_smote, y_prod_smote = run_model(X_train_smote, X_test_smote, y_train_smote, y_test_smote, test_case_ids_smote, True)

print_evaluated_model_accuracy(y_test_smote, y_pred_smote)

Accuracy: 0.97


## Model cross validation

In [34]:
def run_cross_validation(X: pd.DataFrame, y: pd.Series, y_test: pd.Series, y_pred: pd.Series, y_prob: pd.Series, is_smote: bool) -> pd.DataFrame:
    metrics: pd.DataFrame = get_cross_validation_metrics(model, X, y, cv=5)
    test_metrics = get_metrics(y_test, y_pred, y_prob)
    test_metrics["fold"] = 0 # Initial test metrics (before cross validation)
    test = pd.DataFrame([test_metrics])
    test.set_index("fold", inplace=True)

    print_validated_model_accuracy(model, metrics)

    # Prepend test_metrics to metrics dataframe, export and display
    metrics = pd.concat([test, metrics])
    metrics.to_csv(f"../Data/model_metrics_{variant}_{GENE_FILE_VARIANT}{'_smote' if is_smote else ''}.csv", index=False)
    return metrics

In [35]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metrics = run_cross_validation(X, y, y_test, y_pred, y_prod, False)
metrics

Model validation for RandomForestClassifier:
[0.9336734693877551, 0.9336734693877551, 0.9487179487179487, 0.9282051282051282, 0.9384615384615385]

Mean accuracy: 0.9365



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.954082,0.826087,0.791667,0.808511,0.985046,19,168,5,4
1,0.933673,0.695652,0.727273,0.711111,0.967328,16,167,6,7
2,0.933673,0.652174,0.750000,0.697674,0.971350,15,168,5,8
3,0.948718,0.652174,0.882353,0.750000,0.982053,15,170,2,8
4,0.928205,0.608696,0.736842,0.666667,0.975101,14,167,5,9
5,0.938462,0.695652,0.761905,0.727273,0.929221,16,167,5,7


## Smote applied

In [36]:
# Still getting warning:
#   ConvergenceWarning: lbfgs failed to converge (status=1):
#                       STOP: TOTAL NO. OF F,G EVALUATIONS EXCEEDS LIMIT.
metric_smote = run_cross_validation(X_smote, y_smote, y_test_smote, y_pred_smote, y_prod_smote, True)
metric_smote

Model validation for RandomForestClassifier:
[0.9710144927536232, 0.9681159420289855, 0.9710144927536232, 0.9710144927536232, 0.9680232558139535]

Mean accuracy: 0.9698



,accuracy,recall,precision,f1_score,roc_auc,true_positive,true_negative,false_positive,false_negative
fold,,,,,,,,,
0,0.973913,1.000000,0.950276,0.974504,0.997328,172,164,9,0
1,0.971014,1.000000,0.945055,0.971751,0.993480,172,163,10,0
2,0.968116,0.988372,0.949721,0.968661,0.990977,170,164,9,2
3,0.971014,0.976879,0.965714,0.971264,0.994909,169,166,6,4
4,0.971014,0.994220,0.950276,0.971751,0.994371,172,163,9,1
5,0.968023,0.988372,0.949721,0.968661,0.995707,170,163,9,2
